<a href="https://colab.research.google.com/github/sanketmohanty09/CovidDetection/blob/main/CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Jun 22 07:31:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_path='/content/drive/MyDrive/CovidDetection /Train'
test_path='/content/drive/MyDrive/CovidDetection /Val'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
### CNN Based Model
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',metrics=['accuracy'],loss=keras.losses.binary_crossentropy)



In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [ ]:
train_datagen=image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/CovidDetection /Train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 344 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'NORMAL': 1}

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    '/content/drive/MyDrive/CovidDetection /Val',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
    
)

Found 64 images belonging to 2 classes.


In [ ]:
hist=model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=2
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 189s 18s/step - loss: 2.4084 - accuracy: 0.5311 - val_loss: 0.7094 - val_accuracy: 0.3750
Epoch 2/10
8/8 [==============================] - 33s 4s/step - loss: 0.6890 - accuracy: 0.5711 - val_loss: 0.6676 - val_accuracy: 0.7812
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: 0.5563 - accuracy: 0.7676 - val_loss: 0.5141 - val_accuracy: 0.8438
Epoch 4/10
8/8 [==============================] - 12s 2s/step - loss: 0.3902 - accuracy: 0.8290 - val_loss: 0.6119 - val_accuracy: 0.7812
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.2795 - accuracy: 0.9190 - val_loss: 0.6578 - val_accuracy: 0.8125
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.2024 - accuracy: 0.9429 - val_loss: 0.7030 - val_accuracy: 0.8125
Epoch 7/10
8/8 [==============================] - 12s 2s/step - loss: 0.1886 - accuracy: 0.9462 - val_loss: 0.9357 - val_accuracy: 0.8281
Epoch 8/10
8/8 [================